In [1]:
# import
import getdata
import pickle
import numpy as np 
import autoencoder as model
import matplotlib.pyplot as plt
import scipy.ndimage
import cv2

Using TensorFlow backend.


In [3]:
# load training data
flatten = False      #False for 2D images, True for flattened ones
denoise_only = False #False for 128x128 output, True for 64x64 output
rescale = True       #True for [0,255] data, False for [1,0] data
amount = 0           #amount of training data to load, 0 for all data
train_x, train_y = getdata.get_training(flatten = flatten, amount = amount, rescale = rescale, denoise_only = denoise_only)
test_data = getdata.get_test(flatten = flatten, amount = amount, rescale = rescale, denoise_only = denoise_only) 

In [ ]:
# show images if flatten = True
n=0
for data in train_x[n:n+2]:
    size = int(np.shape(data)[0]**(1/2))
    plt.imshow(data.reshape(size,size), cmap = 'Greys')
    plt.show()

In [ ]:
# show images if flatten = False
n=0
for data in train_y[n:n+4]:
    data = data[:,:,0]
    plt.imshow(data, cmap = 'Greys')
    plt.show()

In [6]:
# create network architecture
# test 0 is autoencoder
# test 1 is small convolutional network
# test 2 is large convolutional network
# test 3 is a deconvolutional network which scales up
# Since we are using a large convolutional network, do not adjust the following line:
autoencoder = model.Autoencoder([(64,64,1),(4,3)], (4,3), [(1,3)], test = 4)
print("finished building autoencoder")

finished building autoencoder


In [7]:
# (1) #
# TRAIN THE AUTOENCODER NORMALLY
autoencoder.train(train_x,train_y, epochs = 4, verbose = 1, loss = 'mean_squared_error', optimizer= 'adadelta', batch_size = 10)

AttributeError: 'Autoencoder' object has no attribute 'autoencoder'

In [ ]:
# Run test if Flatten = True

# load and show custom image
user_array = scipy.ndimage.imread('train.png', flatten=True, mode=None)
user_array = 1-(user_array/225)
shape = np.shape(user_array)
imgplot = plt.imshow(user_array, cmap = 'Greys')
plt.show()
# compute and show reconstruction
input_vector = user_array.flatten()
output_vector = autoencoder.predict(np.array([input_vector, input_vector]))[0]
output_array = output_vector.reshape(64,64)
imgplot = plt.imshow(output_array, cmap = 'Greys')
plt.show()

In [ ]:
# TEST NETOWORK for flatten = False

# image number [1...5]
num = 5
# load and show custom image
user_array = scipy.ndimage.imread('x' + str(num) + '.png', flatten=True, mode=None)
user_array = 1-(user_array/225)
imgplot = plt.imshow(user_array, cmap = 'Greys')
plt.show()
# load and show target image
target = scipy.ndimage.imread('y'+ str(num) + '.png', flatten=True, mode=None)
target = 1-(target/225)
if denoise_only:
    target = cv2.resize(target,(64,64))
imgplot = plt.imshow(target, cmap = 'Greys')
plt.show()
# compute and show reconstruction
input_vector = np.expand_dims(user_array, axis = 3)
output_vector = autoencoder.predict(np.array([input_vector, input_vector]))[0]
output_array = output_vector[:,:,0]
imgplot = plt.imshow(output_array, cmap = 'Greys')
plt.show()

In [ ]:
# NOT WORKING CURRENTLY

# (2) #
# INCREMENTALY TEST ON MNIST DATA
import mnist_loader
# load training data
num_digits = 10
val_data = np.array(list(map(lambda x: np.squeeze(x), np.array(list(validation_data))[:,0])))
np.random.shuffle(val_data)
val_data = val_data[0:num_digits]

In [ ]:
# NOT WORKING CURRENTLY

# TRAIN AUTOENCODER AND PRINT RESULT
epochs = 20

plt.figure(figsize=(2*num_digits, 4))
for i in range(num_digits):
    # display original
    ax = plt.subplot(2, num_digits, i + 1)
    plt.imshow(val_data[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

for epoch in range(epochs):
    model.train(training_data, test_data, epochs = 1, verbose = 0)
    predicted_imgs = autoencoder.predict(val_data)
    # copypasta code to display digits
    plt.figure(figsize=(2*num_digits, 4))
    for i in range(num_digits):
        # display reconstruction
        ax = plt.subplot(2, num_digits, i + 1 + num_digits)
        plt.imshow(predicted_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

plt.figure(figsize=(2*num_digits, 4))
for i in range(num_digits):
    # display original
    ax = plt.subplot(2, num_digits, i + 1)
    plt.imshow(val_data[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
    
print("finished training!")

In [ ]:
# SAVE MODEL

import os
from keras.models import model_from_json
save_path = os.getcwd() + '/model_weights/deconv2.h5'

autoencoder.autoencoder.save_weights(save_path)
with open(os.getcwd() + '/model_weights/model_architecture_deconv2.json', 'w') as f:
    f.write(autoencoder.autoencoder.to_json())

In [ ]:
# RELOAD MODEL

# model reconstruction from JSON
with open(os.getcwd() + '/model_weights/model_architecture3.json', 'r') as f:
    autoencoder = model_from_json(f.read())
# load weights into the model
autoencoder.load_weights(os.getcwd() + '/model_weights/model_v3.h5')

autoencoder.summary()